In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [3]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 88% 106M/121M [00:00<00:00, 131MB/s] 
100% 121M/121M [00:00<00:00, 137MB/s]


In [4]:
from zipfile import ZipFile

file_name = '/content/face-expression-recognition-dataset.zip'


# Extract the contents of the zip file
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Data has been extracted')


Data has been extracted


In [6]:
TRAIN_DIR = '/content/images/train'
TEST_DIR = '/content/images/test'

In [7]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [8]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
neutral completed
happy completed
fear completed
sad completed
surprise completed


In [9]:
print(train)

                                          image     label
0          /content/images/train/angry/9041.jpg     angry
1         /content/images/train/angry/15112.jpg     angry
2         /content/images/train/angry/35553.jpg     angry
3           /content/images/train/angry/233.jpg     angry
4         /content/images/train/angry/32841.jpg     angry
...                                         ...       ...
28816  /content/images/train/surprise/34583.jpg  surprise
28817  /content/images/train/surprise/26086.jpg  surprise
28818  /content/images/train/surprise/35285.jpg  surprise
28819  /content/images/train/surprise/24890.jpg  surprise
28820  /content/images/train/surprise/26186.jpg  surprise

[28821 rows x 2 columns]


In [10]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
neutral completed
happy completed
fear completed
sad completed
surprise completed


In [11]:
print(test)
print(test['image'])

                                        image     label
0        /content/images/test/angry/13334.jpg     angry
1        /content/images/test/angry/26446.jpg     angry
2        /content/images/test/angry/35514.jpg     angry
3        /content/images/test/angry/26264.jpg     angry
4        /content/images/test/angry/33459.jpg     angry
...                                       ...       ...
7061   /content/images/test/surprise/8489.jpg  surprise
7062  /content/images/test/surprise/10813.jpg  surprise
7063  /content/images/test/surprise/20407.jpg  surprise
7064  /content/images/test/surprise/22305.jpg  surprise
7065  /content/images/test/surprise/24668.jpg  surprise

[7066 rows x 2 columns]
0          /content/images/test/angry/13334.jpg
1          /content/images/test/angry/26446.jpg
2          /content/images/test/angry/35514.jpg
3          /content/images/test/angry/26264.jpg
4          /content/images/test/angry/33459.jpg
                         ...                   
7061     /conte

In [12]:
from tqdm.notebook import tqdm

In [13]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [14]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [15]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [16]:
x_train = train_features/255.0
x_test = test_features/255.0

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [19]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [20]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [21]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [22]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [23]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
226/226 [==============================] - 22s 58ms/step - loss: 1.8215 - accuracy: 0.2448 - val_loss: 1.7970 - val_accuracy: 0.2624
Epoch 2/100
226/226 [==============================] - 11s 50ms/step - loss: 1.7839 - accuracy: 0.2570 - val_loss: 1.7261 - val_accuracy: 0.2920
Epoch 3/100
226/226 [==============================] - 11s 50ms/step - loss: 1.6682 - accuracy: 0.3281 - val_loss: 1.5104 - val_accuracy: 0.4082
Epoch 4/100
226/226 [==============================] - 11s 50ms/step - loss: 1.5325 - accuracy: 0.3986 - val_loss: 1.4239 - val_accuracy: 0.4465
Epoch 5/100
226/226 [==============================] - 11s 50ms/step - loss: 1.4569 - accuracy: 0.4377 - val_loss: 1.3464 - val_accuracy: 0.4816
Epoch 6/100
226/226 [==============================] - 11s 50ms/step - loss: 1.4012 - accuracy: 0.4613 - val_loss: 1.3077 - val_accuracy: 0.4948
Epoch 7/100
226/226 [==============================] - 11s 50ms/step - loss: 1.3702 - accuracy: 0.4742 - val_loss: 1.2430 - val_ac

In [24]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
